In [1]:
import requests
from tabulate import tabulate

In [2]:
url = 'https://nylottery.ny.gov/drupal-api/api/v2/scratch_off_data?_format=json'
r = requests.get(url)
data  = r.json()

In [11]:
Master_Dictionary = {}
for TicketType in data['rows']:
    Title = TicketType['game_number']
    Master_Dictionary[Title]= {}
    Master_Dictionary[Title]['game_number'] = TicketType['game_number']
    Master_Dictionary[Title]['title'] = TicketType['title']
    All_tier_prices = TicketType['odds_prizes']
    
    overall_odds = TicketType['overall_odds']
    overall_odds = overall_odds.replace('\t',' ')
    overall_odds = overall_odds.replace(':','')
    overall_odds = overall_odds.replace('  ',' ')
    overall_odds = overall_odds.replace('Cash Odds','')
    overall_odds = overall_odds.replace('Odds of Winning Cash Prize','')
     
    Master_Dictionary[Title]['overall_odds'] = overall_odds

    ticket_price = float(TicketType['ticket_price'])
    Master_Dictionary[Title]['ticket_price'] = ticket_price

    Max_overall_odds = All_tier_prices[0]['overall_odds']
    Max_overall_odds = Max_overall_odds.replace('1 in ','')
    Max_overall_odds = Max_overall_odds.replace(',','')
    try:
        Max_overall_odds = float(Max_overall_odds)
    except Exception as err:
        print(err)
        Max_overall_odds = 0 
    Max_prizes_amount = int(All_tier_prices[0]['prizes_paid_out']) + int(All_tier_prices[0]['prizes_remaining'])
    Total_tickets_made = Max_prizes_amount*Max_overall_odds
    
    prizes_remaining = 0
    prizes_paid_out = 0
    for Different_tier_prices in All_tier_prices:
        prizes_remaining = prizes_remaining + int(Different_tier_prices['prizes_remaining'])
        prizes_paid_out = prizes_paid_out + int(Different_tier_prices['prizes_paid_out'])
        
    Perc_of_tickets_Remaining = prizes_remaining/(prizes_remaining+prizes_paid_out)
    Calc_Odds_of_Winning = round((Perc_of_tickets_Remaining*Total_tickets_made)/prizes_remaining,2)
    Calc_Odds_of_Winning = '1 in {}'.format(Calc_Odds_of_Winning)
    
    Master_Dictionary[Title]['Calc_Odds_of_Winning'] = Calc_Odds_of_Winning
    
    Total_Profit_Sum = 0 
    for Different_tier_prices in All_tier_prices:
        Tier_prizes_remaining = int(Different_tier_prices['prizes_remaining'])
        Tier_prizes_paid_out = int(Different_tier_prices['prizes_paid_out'])
        Tier_Perc_of_tickets_Remaining = Tier_prizes_remaining/(Tier_prizes_remaining+Tier_prizes_paid_out)
        Applied_Factor = Tier_Perc_of_tickets_Remaining/Perc_of_tickets_Remaining
        
        string = Different_tier_prices['overall_odds']
        string = string.replace('1 in ','')
        string = string.replace(',','')
        
        try:
            Odds = 1/(float(string)/Applied_Factor)
        except:
            Odds = 0

        prize_amount = Different_tier_prices['prize_amount']
        prize_amount = prize_amount.replace(',','')
        prize_amount = prize_amount.replace('$','')
        try:
            prize_amount = int(prize_amount)
        except:
            prize_amount= 0
        Price_tier_Probability_of_Profit = prize_amount*Odds
        Total_Profit_Sum = Total_Profit_Sum + Price_tier_Probability_of_Profit
        
    Expected_return_perc = (Total_Profit_Sum/ticket_price)-1
    Expected_return_perc  = round(Expected_return_perc*100,2)
    Master_Dictionary[Title]['Expected_return_perc'] = '{}%'.format(Expected_return_perc)
    Master_Dictionary[Title]['Expected_return_perc'] = Expected_return_perc
    
    Master_Dictionary[Title]['Perc_of_tickets_Remaining'] = round(Perc_of_tickets_Remaining*100,2)
    

Header = ['Title', 'Expected return %', 'Overall Odds', 'Calc Odds', 'Price', '% Remaining', 'Ticket Num']
Data_Set = []
for items in Master_Dictionary:
    lottery_Ticket = [Master_Dictionary[items]['title'], Master_Dictionary[items]['Expected_return_perc'], Master_Dictionary[items]['overall_odds'], Master_Dictionary[items]['Calc_Odds_of_Winning'], Master_Dictionary[items]['ticket_price'], Master_Dictionary[items]['Perc_of_tickets_Remaining'], Master_Dictionary[items]['game_number'] ]
    Data_Set.append(lottery_Ticket)

Data_Set.sort(key = lambda i: i[1])
Data_Set.reverse()

Improved_Data = []
Improved_Data.append(Header)
for items in Data_Set:
    Improved_Data.append(items)


print(tabulate(Improved_Data, headers='firstrow'))
### Copy and Paste into notepad to print

Title                             Expected return %  Overall Odds    Calc Odds      Price    % Remaining    Ticket Num
------------------------------  -------------------  --------------  -----------  -------  -------------  ------------
$5,000,000 RICHES                             -4.96  1 in 3.35       1 in 3.35         20           4.74          1483
$10,000,000 BLACK TITANIUM                   -20.71  1 in 3.53       1 in 3.53         30          12.35          1406
MYSTERY MULTIPLIER CASHWORD                  -21.26                  1 in 4.18          3          50.91          1530
200X                                         -24.33  1 in 3.64       1 in 3.64         30          16.73          1468
VIP MILLIONS                                 -25.12  1 in 3.59       1 in 3.59         30          85.89          1546
CASH X100                                    -25.18  1 in 3.68       1 in 3.68         20          89.4           1558
X SERIES: 200X                               -25

In [4]:
# Master_Dictionary

In [5]:
# Max_overall_odds = All_tier_prices[0]['overall_odds']
# Max_overall_odds = Max_overall_odds.replace('1 in ','')
# Max_overall_odds = Max_overall_odds.replace(',','')
# try:
#     Max_overall_odds = float(Max_overall_odds)
# except Exception as err:
#     print(err)
#     Max_overall_odds = 0 
# Max_prizes_amount = int(All_tier_prices[0]['prizes_paid_out']) + int(All_tier_prices[0]['prizes_remaining'])
# Total_tickets_made = Max_prizes_amount*Max_overall_odds


# Total_tickets_made

In [6]:
# Master_Dictionary['1453']